Reflector Area Processing - Further Analysis
============================================



The analysis in "Reflector Area Processing - Testing" revealed that there are significant differences in all metrics except for area distribution shapes when comparing a part of a raw section versus the same part of section that had fractured grains (both actually-fractured as well as split up due to the thresholding method) manually reconstructed. These differences also signify that the raw data is not the most representative of actual reflector distributions, and hence some sort of gap-filling (whether manual or automatic) is required.

Due to the excessive time consumption of this manual gap filling, automatic gap filling is preferred for the full analysis. However the full analysis in `../section-scans-full/working.org` showed that there's the degree of automatic gap filling (via dilation-erosion) affects the outcome as well. In order to evaluate the different degrees of automatic gap filling, the following analysis is devised:

1.  Display the reflector patches without gap filling, with reflector patches of interest (i.e. with area greater than or equal to 5 px and lesser than or equal to 0.05 mm<sup>2</sup>) in one colour and the others in another colour.
2.  Run the automatic gap filling.
3.  Filter out the reflectors of interest after gap filling.
4.  Display the reflector patches of interest after automatic gap filling in one colour, and the other reflector patches after automatic gap filling in another colour.
5.  Visually inspect the difference in reflector patches before and after to see whether the automatic gap filling produces reasonable results (reconstructs a large number of grains that were clearly whole originally without artificially creating too many nonsensical large grains from grains that are clearly separate).



## Contour Area Slicing



This involves converting the contour area slicing method to display just contours with areas of interest described in `../section-scans-example/working.org` into a reusable method.

Since area distributions are of the greatest focus, a section whose area distribution changed significantly with increasing (automatic) dilation-erosion (see `../section-scans-full/working.org`) was selected for an initial run of this analysis: M01.



In [1]:
# Options to change.
sample = "M01"
dilate_erode = 10

from area_slicer import *
import os

import sys
sys.path.insert(0,"../section-scans-full")
from util_funcs import *

pix2mm = 0.001

AS = AreaSlicer(sample,dilate_erode)
img = AS.visualise_patches([0,5*pix2mm**2],(255,0,0))
img = AS.visualise_patches([0.05,None],(255,0,0),img)
img = AS.visualise_patches([5*pix2mm**2,0.05],(0,255,0),img)

if not os.path.exists(f"./imgs/{sample}-{str(dilate_erode)}-sliced.jpg"):
    cv2.imwrite(f"./imgs/{sample}-{str(dilate_erode)}-sliced.jpg",img)

Features of interest from the results are shown in figure [8](#org28c7057).

-   Well joined region:
    -   Where there's no dilation-erosion: reflector patches that are interpreted as separate and appear separate.
    -   Where there's some dilation-erosion: reflector patches that are interpreted as being one patch and appear as such. Embayed regions may or may not be filled in.
-   Badly joined region:
    -   Where there's no dilation-erosion: reflector patches that are interpreted as being one patch but appear separate.
    -   Where there's some dilation-erosion: reflector patches that are joined without being interpreted as being one patch, or are "well-joined" patches that have grown beyond the borders of the interpreted patch.

![img](./imgs/comparisons.png "Comparing the outputs of different degrees of automatic gap filling for parts of sample M01, showing a region that's been gap-filled well, and a region that's been very poorly gap-filled.")

Using a 10x10 px kernel for dilation-erosion appears to produce the best results, with individual patches being joined up without overjoining. However, there are still cases of overjoining, often with narrow bridges between grains that should be separate.

One major issue contributing to this is the joining of grains that have been defined as "noise" on the basis of their area covering less than 5 px (irrespective of whether they actually are or not). The result of this is the appearance of false grains - grains that enter the realm of signal (on the basis of area growing greater than 5 px) due to size growth related to the dilation-erosion process.

Therefore, it's proposed that grains interpreted as noise on the basis of their size are discarded **before** the dilation-erosion process takes place. This order is more sensible than previous since anything interpreted as "noise" shouldn't be processed as part of the actual signal. Furthermore, the dilation-erosion that should produce the best results (out of the ones tested) uses a 10x10 px kernel.

This requires a relatively major change to the processing's command flow, as captured by the following code:



In [1]:
dilate_erode = 10

def load_unmerged_contours(sample):
    contours = np.load(f"../../DATASETS/RL_scans/contours-unmodified/{sample}.png.npy",allow_pickle=True)
    larger_contours = np.load(f"../../DATASETS/RL_scans/contours-unmodified/{sample}.png-larger.npy",allow_pickle=True)
    return contours,larger_contours

def find_areas(contours,larger_contours,pix2mm=None):
    contour_areas = np.array(list(map(cv2.contourArea,contours)))
    larger_contour_areas = np.array(list(map(cv2.contourArea,larger_contours)))
    patch_areas = (larger_contour_areas + 1 - 2 * contour_areas)/2
    if pix2mm:
        patch_areas = np.array(patch_areas) * (pix2mm**2)
        units = "mm"
    else:
        units = "px"
    return patch_areas,units

def draw_contours(contours,img):
    [cv2.fillPoly(img,[np.reshape(c,(c.shape[0],2))],color=(255,255,255,255)) for c in contours]
    return img

if not os.path.exists(f"./imgs/{str(dilate_erode)}-{sample}.png"):
    contours,larger_contours = load_unmerged_contours(sample)
    areas,units = find_areas(contours,larger_contours,pix2mm)
    size_filter = construct_minmax_filter(areas,5*pix2mm**2)
    contours = list_of_list_filter(contours,size_filter)
    img = cv2.imread(f"../../DATASETS/RL_scans/{sample}.jpg")
    blank_img = np.zeros([*img.shape[:2],4])
    contour_img = draw_contours(contours,blank_img)
    cv2.imwrite(f"./imgs/{str(dilate_erode)}-{sample}.png",contour_img)
else:
    img = cv2.imread(f"../../DATASETS/RL_scans/{sample}.jpg")

AP = AreaProcessor(f"./imgs/{str(dilate_erode)}-{sample}.png",pix2mm,dilate_erode)
contours,larger_contours = AP.load_contours()

AS = AreaSlicer(sample,dilate_erode,base_folder="./imgs",AP=AP)
img = AS.visualise_patches([0.05,None],(255,0,0),img)
img = AS.visualise_patches([5*pix2mm**2,0.05],(0,255,0),img)

if not os.path.exists(f"./imgs/refined-{sample}-{str(dilate_erode)}-sliced.jpg"):
    cv2.imwrite(f"./imgs/refined-{sample}-{str(dilate_erode)}-sliced.jpg",img)

Features of interest from the refined results are shown in figure [2](#org2f10c5e).

![img](./imgs/refined-comparisons.png "Comparing the outputs of different degrees of automatic gap filling for parts of sample M01 when using the refined processing pathway, showing the same regions as in the previous figure.")

This new method not only reduces overjoining and development of badly-joined regions (in both 10x10 px and 20x20 px kernel dilation-erosion cases), but also avoids joining "noise" and creating false reflectors. In the top row, 10x10 px kernel dilation-erosion doesn't fully reconstruct the coarsest grain like 20x20 px, but that grain is very heavily embayed, and 10x10 px has the most well joined patches in the bottom row. Despite the issues with a 20x20 px kernel, visually comparing the bottom-right image (new method) in this image to the previous (old method) also reveals a significant improvement.

This refined method and a 10x10 px kernel will be used in `../section-scans-refined/working.org` to produce confidently-interpretable results.

